In [1]:
import healpy as hp
import numpy as np
import burstfuncs
import random as rand
import statistics as s
import burstfuncs as bf

This is my workspace where I'm experimenting with the class functions before I end up transferring it into the py version. Some of the things I still need to do while here is:


Documentation

Random gauss vs poisson

Importing modules 

 adding skymaps. 
 
 
 Updating how classes are defined (input really best approach?) 

After redefining classes, build up BurstCube class so that it runs the entire sim! call on module that contains all the functions, length, angle, sep, etc. Should be easiest this way

In [2]:
class GRBs():
   #this produces an error, deal with soon!
    """
    Generates an array of GRB's given 
    certains strength at different sky positions.
    
    Output should be an array. 
    """
    
  #  import numpy as np
   # import healpy as hp
    def __init__(self,NSIDE,strength):
        from healpy import nside2npix
        from healpy import pix2ang
        #depending on NSIDE, there will be anywhere from 12 to infinite spots on the sky w/ GRBs
        self.Ao = strength
        self.pixels = nside2npix(NSIDE)

        #want to convert these pixels into theta phi coords. 
        self.sourceangs = []
        for i in range(self.pixels):
            self.sourceangs.append(pix2ang(NSIDE,i))

    def say_Ao(self):
        print("The GRBs being tested will be " + str(self.Ao) + " counts strong.")
        
            

In [3]:
GRB = GRBs(4,500)

In [4]:
GRB.pixels

192

In [5]:
GRB.say_Ao()

The GRBs being tested will be 500 counts strong.


In [13]:
class BurstCube():

    def __init__(self,background):
        import numpy as np
        import healpy as hp
        import burstfuncs as bf
        self.zenith = [0 , 0]
        self.bg = background
        test = input("Are the detectors alternating? ")
        if test == "yes" or test == "Yes" or test == "y" or test == "asdlfkjawe":           
            self.tiltA = np.deg2rad(float(input("Please enter the first tilt (deg) ")))
            self.tiltB = np.deg2rad(float(input("Please enter the second tilt (deg) ")))
            self.tiltC = self.tiltA
            self.tiltD = self.tiltB
        else:
            self.tilt = np.deg2rad(float(input("Please enter the tilt (deg) ")))
            self.tiltA = self.tiltB = self.tiltC = self.tiltD = self.tilt
            
    
    #make the normal vectors!
        #self.detA = [ zenith[0] + self.tiltA , zenith[1] ] 
        #self.detB = [ zenith[0] + self.tiltB , zenith[1] + np.pi/2 ] 
       # self.detC = [ zenith[0] + self.tiltC , zenith[1] + np.pi ] 
        #self.detD = [ zenith[0] + self.tiltD , zenith[1] + 3*np.pi/2 ] 

        #self.Anorm = hp.ang2vec(self.detA[0],self.detA[1])
        #self.Bnorm = hp.ang2vec(self.detB[0],self.detB[1])
        #self.Cnorm = hp.ang2vec(self.detC[0],self.detC[1])
        #self.Dnorm = hp.ang2vec(self.detD[0],self.detD[1])

    
       # self.dets = [self.Anorm,self.Bnorm,self.Cnorm,self.Dnorm] 
        
    @property
    def detA(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltA , self.zenith[1] ]
    @property 
    def detB(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltB , self.zenith[1] + np.pi/2 ]
    @property
    def detC(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltC , self.zenith[1] + np.pi]
    @property 
    def detD(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltD , self.zenith[1] + 3*np.pi/2 ]
    @property
    def normA(self):
        return  hp.ang2vec(self.detA[0],self.detA[1])
    @property 
    def normB(self):
        return  hp.ang2vec(self.detB[0],self.detB[1])
    @property
    def normC(self):
        return  hp.ang2vec(self.detC[0],self.detC[1])
    @property 
    def normD(self):
        return  hp.ang2vec(self.detD[0],self.detD[1])

    
    @property
    def dets(self):
        return [self.normA,self.normB,self.normC,self.normD] 
    
    
    
    def response2GRB(self, GRB):   #is this how I inherit? 
        #first need to include the GRB.
       
        """
        Using least squares regression, respond2GRB will determine the sky position of an array of GRB sources assuming some inherent background noise within 
        detectors, along with fluctuations of either Gaussian or Poissonian nature. 

        Parameters
        ----------
        GRB : object
        
        An instance of the separately defined "GRBs" class that contains a number of evenly spaced sky positions of a given strength. 

        Returns
        ----------
        localizationerrors : array

        numpy array that contains the average localization uncertainty at each sky position. 

        
        """
        
        self.localizationerrors = []    
        for i in range(len(GRB.sourceangs)):
            sourceAng = GRB.sourceangs[i]
            print("Testing " + str(np.rad2deg(sourceAng)))
           #this check passes.       

            
           # print("Testing at " + str(np.rad2deg(GRB.sourceangs)))
            sourcexyz = hp.ang2vec(sourceAng[0],sourceAng[1]) #cartesian position of the burst
            loop = 0 #I'm going to want to sample each sky position more than once,
                    #here's where I define how many times that is
            locunc = []
            while loop<5:
                detcounts = []  #number of counts incident in each detector. 
                for j in range(len(self.dets)):
                    sep=bf.angle(sourcexyz,self.dets[j])
                   # print("separation is " + str(np.rad2deg(sep)))
                   #this check passes.  
               
                    if sep < np.pi/2: # meaning if >90, would not be facing detector.
                        dtheory=GRB.Ao*bf.response(sourcexyz,self.dets[j])  #still need to define strength, brb and gonna do that 
                    else: #like I was saying, has to face it!
                        dtheory = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                    counts = dtheory + self.bg #another artifact, incl this background effect somewhere
                    unccounts = np.sqrt(counts)
                    detactual = rand.gauss(counts,unccounts)  #there is a lot of noise, present, updating it now. 
                    if detactual-self.bg < 0:
                        detactual = 0
                    
                    detcounts.append(detactual)
                
                
                coarsethetaloc,coarsephiloc,coarseAo = bf.solver(detcounts,self.dets,0,90,0,360,10,37,self.bg)
                finethetaloc,finephiloc,fineAo = bf.solver(detcounts,self.dets,coarsethetaloc-5,coarsethetaloc+5,coarsephiloc-5,coarsephiloc+5,10,10,self.bg)
                
                if finethetaloc > 180:
                #    print("it recovered an unrealistic answer, skip")
                    break
                elif finethetaloc < 0:
                    #print("Same issue, there are limits to theta that are broken here. ")
                    break 
                    
                recpos = [finethetaloc,finephiloc]
              #  print("Recovered position at " + str(recpos))
                
                recvec = hp.ang2vec(np.deg2rad(finethetaloc),np.deg2rad(finephiloc))
                locunc.append(bf.angle(sourcexyz,recvec))
               # print("loc unc: " + str(np.rad2deg(bf.angle(sourcexyz,recvec))) + " deg")
                loop+=1
          #  print("Obtained avg loc unc of " + np.rad2deg(s.mean(locunc)) + "")
            if len(locunc)>0:
                self.localizationerrors.append(np.rad2deg(s.mean(locunc)))
            else:
                self.localizationerrors(1000)
                #some big # signifying that it can't catch it 

            print("Loc unc " +  str(np.rad2deg(s.mean(locunc))))
                #loc unc is the uncertainty at each sky position, localerros is for all of them
            
            #should there be more selfs? more prints? basically done so time to debug after lunch. 
        
        maybe = input("All done with the simulation, want to make the skymap too? ")
        
        if maybe == "yes" or maybe == "Yes" or maybe == "y": 
            import matplotlib.pyplot as plt
            from healpy import newvisufunc
            if len(self.localizationerrors) == len(GRB.sourceangs):
                im = np.array(self.localizationerrors)
            else:
                blockedpart=1000*np.ones(GRB.pixels-len(self.localizadtionerrors))
                im = np.concatenate((self.localizationerrors,blockedpart))
            hp.newvisufunc.mollview(im,min=0, max=30,unit='Localization Accurary (degrees)',graticule=True,graticule_labels=True,cmap='viridis_r')
            plt.title('All Sky Localization Uncertainty for BurstCube')
        else:
            print("Ok, maybe next time!")

        return self.localizationerrors
            
    
    

In [ ]:
    def plot_skymap(self,GRB):
        import numpy as np
        import matplotlib.pyplot as plt
        import healpy as hp
        from healpy import newvisufunc
        if len(self.localizationerrors) == len(GRB.sourceangs):  #if the function successfully was able to catch all the spots (meaning nothing went wrong!)
            im = np.array(self.localizationerrors) 
        else:
            blockedpart=1000*np.ones(GRB.pixels-len(angoffset))
            im=np.concatenate((self.localizationerrors,blockedpart))
        hp.newvisufunc.mollview(im,min=0, max=50,unit='Localization Accurary (degrees)',graticule=True,graticule_labels=True,cmap='viridis_r')
        plt.title('All Sky Localization Uncertainty for BurstCube')
        

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
from healpy import newvisufunc
%matplotlib notebook
if len(im) == 48:
    im = np.array(im)
else:
    blockedpart= 1000*np.ones(48-len(im))
    im=np.concatenate((im,blockedpart))
hp.newvisufunc.mollview(im,min=0, max=50,unit='Localization Accurary (degrees)',graticule=True,graticule_labels=True,cmap='viridis_r')


In [ ]:
im = [12.193725149835284,
 25.25448240996645,
 22.305707106859245,
 23.965693073949609,
 52.008548584668084,
 27.821629706579458,
 37.423685868114056,
 68.476263273719425,
 30.625361189430574,
 23.401815051293706,
 43.663123284390203,
 19.564729694837329,
 51.433859688574401,
 74.844783683652736,
 56.576057682336121,
 33.323870897313476,
 90.612650101315353,
 82.253008561973189,
 80.329406453087046,
 94.19026553811679,
 99.538349117642525,
 75.175862782099529,
 51.048488219348087,
 74.128524449751453,
 58.480977383443751,
 52.597647136484362,
 77.222734640119555,
 103.61494510174424,
 41.109554839392487,
 76.269478377179141,
 98.202392939060829,
 99.73653357751094,
 122.38781518900511,
 74.733648939874953,
 99.483079370711494,
 100.81648250231294,
 142.5793551632658,
 128.99519400895949,
 109.03771544592294,
 98.612082515419004,
 127.9138150277825,
 93.497546429808551,
 141.23279694489702,
 130.06560240843098,
 157.61167766859899,
 129.80910207180099]


In [ ]:
class Plots:

In [14]:
cube1 = BurstCube(background=1000)

Are the detectors alternating? yes
Please enter the first tilt (deg) 30
Please enter the second tilt (deg) 45


In [15]:
cube1.response2GRB(GRB)

Testing [ 11.71585239  45.        ]
Loc unc 7.83707702803
Testing [  11.71585239  135.        ]
Loc unc 7.10886232712
Testing [  11.71585239  225.        ]
Loc unc 8.69627887305
Testing [  11.71585239  315.        ]
Loc unc 6.29350423088
Testing [ 23.55646431  22.5       ]
Loc unc 7.80008825631
Testing [ 23.55646431  67.5       ]
Loc unc 7.83651625117
Testing [  23.55646431  112.5       ]
Loc unc 5.35920109952
Testing [  23.55646431  157.5       ]
Loc unc 7.84279692368
Testing [  23.55646431  202.5       ]
Loc unc 12.5738173262
Testing [  23.55646431  247.5       ]
Loc unc 7.48369848419
Testing [  23.55646431  292.5       ]
Loc unc 5.11094215549
Testing [  23.55646431  337.5       ]
Loc unc 7.28411673777
Testing [ 35.6590877  15.       ]
Loc unc 8.54165015053
Testing [ 35.6590877  45.       ]
Loc unc 3.93794638915
Testing [ 35.6590877  75.       ]
Loc unc 10.0438770349
Testing [  35.6590877  105.       ]
Loc unc 5.39633731757
Testing [  35.6590877  135.       ]
Loc unc 3.81872153777
Te

KeyboardInterrupt: 

In [ ]:
cube1.plot_skymap()

In [ ]:
import matplotlib.pyplot as plt
plt.show()

In [ ]:
#to generate an array of all the source angles, and basically follow that 

GRB = GRBs(2)  #only thing I have to identify is the nside
len(GRB.sourceangs)

In [ ]:
import numpy as np
import healpy as hp
import burstfuncs as bf
import random as rand

class GRBs():
 
    """
    Generates an array of GRB's with some given strength at different sky positions.
    
    Output should be an array. 
    """
    def __init__(self,NSIDE,strength):
        import numpy as np
        import healpy as hp
        #depending on NSIDE, there will be anywhere from 12 to infinite spots on the sky w/ GRBs
        self.Ao = strength
        self.pixels = hp.nside2npix(NSIDE)

        #want to convert these pixels into theta phi coords. 
        self.sourceangs = []
        for i in range(self.pixels):
            self.sourceangs.append(hp.pix2ang(NSIDE,i))


class BurstCube:
    def __init__(self,background):
        import numpy as np
        import healpy as hp
        import burstfuncs as bf
        self.zenith = [0 , 0]
        self.bg = background
        test = input("Are the detectors alternating? ")
        if test == "yes" or test == "Yes" or test == "y" or test == "asdlfkjawe":           
            self.tiltA = np.deg2rad(float(input("Please enter the first tilt (deg) ")))
            self.tiltB = np.deg2rad(float(input("Please enter the second tilt (deg) ")))
            self.tiltC = self.tiltA
            self.tiltD = self.tiltB
        else:
            self.tilt = np.deg2rad(float(input("Please enter the tilt (deg) ")))
            self.tiltA = self.tiltB = self.tiltC = self.tiltD = self.tilt
            
        
    @property
    def detA(self):
        return [ self.zenith[0] + self.tiltA , self.zenith[1] ]
    @property 
    def detB(self):
        return [ self.zenith[0] + self.tiltB , self.zenith[1] + np.pi/2 ]
    @property
    def detC(self):
        return [ self.zenith[0] + self.tiltC , self.zenith[1] ]
    @property 
    def detD(self):
        return [ self.zenith[0] + self.tiltD , self.zenith[1] + np.pi/2 ]
    @property
    def normA(self):
        return  hp.ang2vec(self.detA[0],self.detA[1])
    @property 
    def normB(self):
        return  hp.ang2vec(self.detB[0],self.detB[1])
    @property
    def normC(self):
        return  hp.ang2vec(self.detC[0],self.detC[1])
    @property 
    def normD(self):
        return  hp.ang2vec(self.detD[0],self.detD[1])

    
    @property
    def dets(self):
        return [self.normA,self.normB,self.normC,self.normD] 
    
    
    
    
    def respond2GRB(self, GRB):   #is this how I inherit? 
        #first need to include the GRB.
        
        #Putting these here as a reminder, need to learn how to implement Gaussian fluctuations, hopefully in a similar to the actions gauss(x,sigmax)
        #Gauss = True
        #Poisson = False


        """
        Using least squares regression, respond2GRB will determine the sky position of an array of GRB sources assuming some inherent background noise within 
        detectors, along with fluctuations of either Gaussian or Poissonian nature. 

        Parameters
        ----------
        GRB : object
        
        An instance of the separately defined "GRBs" class that contains a number of evenly spaced sky positions of a given strength. 

        Returns
        ----------
        localizationerrors : array

        numpy array that contains the average localization uncertainty at each sky position. 

        
        """
        localizationerrors = []    
        for i in range(len(GRB.sourceangs)):
            sourceAng = GRB.sourceangs[i]
           # print("Source angle is " + str(sourceAng))
           #this check passes.       

            
           # print("Testing at " + str(np.rad2deg(GRB.sourceangs)))
            sourcexyz = hp.ang2vec(sourceAng[0],sourceAng[1]) #cartesian position of the burst
            loop = 0 #I'm going to want to sample each sky position more than once,
                    #here's where I define how many times that is
            locunc = []
            while loop<3:
                detcounts = []  #number of counts incident in each detector. 
                for j in range(len(self.dets)):
                    sep=bf.angle(sourcexyz,self.dets[j])
                   # print("separation is " + str(np.rad2deg(sep)))
                   #this check passes.  
               
                    if sep < np.pi/2: # meaning if >90, would not be facing detector.
                        dtheory=GRB.Ao*bf.response(sourcexyz,self.dets[j])  #still need to define strength, brb and gonna do that 
                    else: #like I was saying, has to face it!
                        dtheory = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                    counts = dtheory + self.bg #another artifact, incl this background effect somewhere
                    unccounts = np.sqrt(counts)
                    detactual = rand.gauss(counts,unccounts)  #there is a lot of noise, present, updating it now. 
                    if detactual-self.bg < 0:
                        detactual = 0
                    
                    detcounts.append(detactual)
    
                   #now for the LSF method using chi squared, applies a coarse to fine refactor here. 
                   #plenty of old variable names to update as well. 
                
                   # print("after fluctuations, " + str(detactual))
                    #fixed indentation errors, uninstalled module, legacy code all fixed. Passes. 

                
                
                coarsethetaloc,coarsephiloc,coarseAo = bf.solver(detcounts,self.dets,0,90,0,360,12,self.bg)
                finethetaloc,finephiloc,fineAo = bf.solver(detcounts,self.dets,coarsethetaloc-5,coarsethetaloc+5,coarsephiloc-7,coarsephiloc+7,12,self.bg)
                
                if finethetaloc > 180:
                #    print("it recovered an unrealistic answer, skip")
                    break
                elif finethetaloc < 0:
                    #print("Same issue, there are limits to theta that are broken here. ")
                    break 
                    
                recpos = [finethetaloc,finephiloc]
                recvec = hp.ang2vec(np.deg2rad(finethetaloc),np.deg2rad(finephiloc))
                locunc.append(bf.angle(sourcexyz,recvec))
               # print("loc unc" + str(locunc))
                loop+=1
            print(np.rad2deg(s.mean(locunc)))

            localizationerrors.append(np.rad2deg(s.mean(locunc)))

           # print("obtained error of" +  str(s.mean(locunc)))
                #loc unc is the uncertainty at each sky position, localerros is for all of them
            
            #should there be more selfs? more prints? basically done so time to debug after lunch. 
        return localizationerrors
        

In [ ]:
GRB = GRBs(NSIDE=8,strength=500)

In [ ]:
cubesat = BurstCube(background=1000)

In [ ]:
cubesat.bg

In [ ]:
cubesat.respond2GRB(GRB)